#Graphs & Social Networks &mdash; lab material
This notebook contains a definition of a python class *Graph*, containing functions described during labs.

Package import.

In [2]:
import numpy as np
from random import random, seed
from copy import deepcopy

## Simple functions from the first classes
This is left just as an example (to compare the behaviour).

In [3]:
def print_matrix(vertices, matrix):
  """
  Printing a graph given by adjacency matrix
  """
  n = len(matrix)
  if (vertices is not None) and (len(vertices) == n):
    vv = vertices
  else:
    vv = range(1, n+1)
  for i in range(n):
    print(vv[i], ":", end="")
    for j in range(n):
      if matrix[i, j]:
        print(" ", vv[j], end="")
    print("")

def print_dict(graph):
  """
  Printing of a graph (given as a dictionary/neighbouring list)
  """
  for v in graph:
    print(v, ":", end="")
    for u in graph[v]:
      print(" ", u, end="")
    print("")

## Class *Graph*

In [4]:
class Graph:
    def __init__(self, graph=None):
        if graph is None:
            graph = {}
        self.graph = graph

    # dict initializer
    @classmethod
    def from_dict(cls, graph):
        return cls(graph)

    # array initializer
    @classmethod
    def from_matrix(cls, matrix, vertices = None):
        if (vertices is None) or (len(vertices) != len(matrix)):
            vertices = [*range(1, len(matrix) + 1)]
        return cls.from_dict(cls._matrix_to_dict(matrix, vertices))

    # two private methods matrix <-> dictionaries
    def _matrix_to_dict(matrix, vertices: list) -> dict:
        """
        Converts a graph given as an adjacency matrix to a graph in dict form.
        """
        res_dict = {}
        for i, v in enumerate(vertices):
            neighbours = [vertices[j] for j, edge in enumerate(matrix[i]) if edge]
            res_dict[v] = neighbours
        return res_dict

    def _dict_to_matrix(self, _dict: dict) -> np.array:
        """
        Converts a graph in dict form to its adjacency matrix.
        """
        n = len(_dict)
        vertices = [*_dict.keys()]
        matrix = np.zeros(shape = (n, n), dtype=int)
        for u,v in [
            (vertices.index(u), vertices.index(v))
            for u, row in _dict.items() for v in row
        ]:
            matrix[u][v] += 1
        return matrix

    def vertices(self) -> list:
        """
        Returns list of vertices of the graph.
        """
        return [*self.graph.keys()]

    def matrix(self) -> np.array:
        """
        Returns the adjacency matrix of the graph.
        """
        return self._dict_to_matrix(self.graph)

    # redefinition of print for objects of class Graph
    def __str__(self):
        res = ""
        for v in self.graph:
            res += f"{v}:"
            for u in self.graph[v]:
                res += f" {u}"
            res += "\n"
        return res

    # The following is for free thanks to the above
    def to_neighbourlist(self, filename: str):
        """
        Saves a graphs to a text file as a neighbour dict.\n
        Filename is a file path.
        """
        file = open(filename, "w")  # open textfile for writing
        file.write(str(self))
        file.close()

    # Modyfying graphs
    def add_vertex(self, vertex):
        """
        Adds a new vertex to the graph.
        """
        if vertex not in self.graph:
            self.graph[vertex] = []

    def del_vertex(self, vertex):
        """
        Removes a vertex from the graph.
        """
        if vertex in self.graph:
            self.graph.pop(vertex)
            for u in self.graph:
                if vertex in self.graph[u]:
                    self.graph[u].remove(vertex)

    def add_arc(self, arc):
        """
        Given pair of vertices (arc variable) add an arc to the graph
        We consider simple, directed graphs.
        """
        u, v = arc
        self.add_vertex(u)
        self.add_vertex(v)
        if v not in self.graph[u]:
            self.graph[u].append(v)

    def add_edge(self, edge: list):
        """
        Given pair of vertices (edge variable) add an edge to existing graph.
        We consider simple, undirected graphs, as symmetric digraphs without loops.
        """
        u, v = edge
        if u == v:
            raise ValueError("Loops are not allowed!")
        self.add_vertex(u)
        self.add_vertex(v)
        if v not in self.graph[u]:
            self.graph[u].append(v)
        if u not in self.graph[v]:
            self.graph[v].append(u)

    # reading from a file
    @staticmethod
    def from_edges(filename: str, directed = 0):
        """
        Read the graph from file, that in each line contains either
        the description of a vertex (one word) or
        the description of an edge/arc (at least 2 words).
        The resulting graph is returned as a neighbourhood list.
        Variable "filename" contains the whole path to the file.
        """
        graph = Graph()
        file = open(filename, "r")          # open the file to read
        for line in file:                   # for each line of the file
          words = line.strip().split()      # splits the line into words
          if len(words) == 1:               # one word - vertex description
            graph.add_vertex(words[0])
          elif len(words) >= 2:             # at least two words, first two are the edge description
            if directed:
              graph.add_arc([words[0], words[1]])
            else:
              graph.add_edge([words[0], words[1]])
        file.close()
        return graph

    @staticmethod
    def random_graph(n: int, p: float):
        """
        Creates a random graph in G(n, p) model.
        """
        rand_graph = Graph()
        for i in range(1, n + 1):
            rand_graph.add_vertex(i)
            for j in range(1, i):
                if random() < p:
                    rand_graph.add_edge([i, j])
        return rand_graph

    @staticmethod
    def cycle(n: int):
        """
        Creates a cycle C_n on n vertices
        """
        cycle = Graph()
        for i in range(n-1):
          cycle.add_edge([i+1, i+2])
        cycle.add_edge([1, n])
        return cycle


    def Prufer(self):
      """
      Returns the Prufer code of a tree.
      It is necessary that the graph is a tree (it is not checked).
      Result is given as a string (empty for trees on 1 or 2 vertices).
      """
      tr = deepcopy(self.graph)   # copy of a tree, as we destroy it
      code = ""
      for i in range(len(self.graph) - 2):
        for x in sorted(tr):
          if len(tr[x]) == 1:   # least leaf
            break
        v = tr[x][0]            # the unique neighbour of x
        code = code + f"{v} "
        tr[v].remove(x)         # remove x from neighbours of v
        tr.pop(x)               # remove x from the tree
      return code.strip()

    @staticmethod
    def tree_from_Prufer(code: str):
        """
        Creating a tree from a Prufer code.
        """
        tree = Graph()
        clist = [int(x) for x in code.strip().split()]   # code as a list of numbers
        n = len(clist) + 2                  # number of vertices
        vert = [*range(1, n+1)]             # list of numbers 1..n
        for v in vert:
          tree.add_vertex(v)
        for i in range(n-2):
          for x in vert:
            if not x in clist:    # x - least leaf
              break
          v = clist.pop(0)    # remove the first element from the code - the neighbour of x
          tree.add_edge((x, v))
          vert.remove(x)
        tree.add_edge(vert)
        return tree


    def connected_components(self):
      """
      Looks for connected components of undirected graph.
      Returns a list of its vertex-sets.
      Remark: the first element contains the set of all graph vertices
      """
      def DFS(u):
        """
        Deep first search (as internal method).
        """
        for w in self.graph[u]:
          if w not in VT[0]:      # w - not visited yet
            VT[0].add(w)          # already visited
            VT[-1].add(w)         # w - in the last connected component
            DFS(w)

      """
      VT - list of vertex sets VT[i] for i > 0 - is a vertex set of i-th connected component
      VT[0] - is a vertex set of the spanning forest (or during the algorithm list of visited vertices).
      """
      VT = [set([])]
      for v in self.graph:
        if v not in VT[0]:      # v is not visited
          VT[0].add(v)
          VT.append(set([v]))   # statring point of new conected component
          DFS(v)
      return VT


    def preorder(self, v, visited=None):
        """
        Realiza un recorrido en preorden del árbol, comenzando desde el vértice `v`.
        `visited` es un conjunto que lleva un registro de los vértices ya visitados.
        """
        if visited is None:
            visited = set()  # Inicializa el conjunto de vértices visitados
        visited.add(v)       # Marca el vértice actual `v` como visitado
        print(v, end=" ")    # Imprime el vértice `v` en preorden (antes de visitar hijos)

        for neighbor in self.graph[v]:      # Itera a través de los vecinos de `v`
            if neighbor not in visited:     # Si el vecino no ha sido visitado
                self.preorder(neighbor, visited)  # Llama recursivamente a preorder

    def postorder(self, v, visited=None):
        """
        Realiza un recorrido en postorden del árbol, comenzando desde el vértice `v`.
        `visited` es un conjunto que lleva un registro de los vértices ya visitados.
        """
        if visited is None:
            visited = set()  # Inicializa el conjunto de vértices visitados
        visited.add(v)       # Marca el vértice actual `v` como visitado

        for neighbor in self.graph[v]:      # Itera a través de los vecinos de `v`
            if neighbor not in visited:     # Si el vecino no ha sido visitado
                self.postorder(neighbor, visited)  # Llama recursivamente a postorder

        print(v, end=" ")    # Imprime el vértice `v` en postorden (después de visitar hijos)

    def connected_components_graphs(self):
        """
        Genera una lista de subgrafos (componentes conectadas) del grafo original.
        Usa la función ConnectedComponents para encontrar los componentes conectados.
        """
        components = self.connected_components()  # Obtiene los componentes conectados como conjuntos de vértices
        component_graphs = []                     # Lista para almacenar los subgrafos

        for component in components[1:]:  # Ignora el primer conjunto que contiene todos los vértices visitados
            subgraph = Graph()            # Crea un nuevo grafo para el componente conectado
            for v in component:
                subgraph.add_vertex(v)    # Añade cada vértice al subgrafo
                for neighbor in self.graph[v]:
                    if neighbor in component:
                        subgraph.add_edge([v, neighbor])  # Añade los bordes si están dentro del componente
            component_graphs.append(subgraph)  # Añade el subgrafo a la lista de subgrafos

        return component_graphs  # Devuelve la lista de subgrafos de componentes conectadas

    @staticmethod
    def random_bipartite_graph(m, n, p):
        """
        Genera un grafo bipartito aleatorio con `m` y `n` vértices en cada conjunto de partición,
        y una probabilidad `p` de que cada par de vértices de conjuntos distintos esté conectado.
        """
        bipartite_graph = Graph()  # Crea el grafo bipartito
        part1 = [f"U{i}" for i in range(1, m + 1)]  # Define el conjunto de vértices `U`
        part2 = [f"V{j}" for j in range(1, n + 1)]  # Define el conjunto de vértices `V`

        for u in part1:
            bipartite_graph.add_vertex(u)  # Añade cada vértice en `U`
        for v in part2:
            bipartite_graph.add_vertex(v)  # Añade cada vértice en `V`

        for u in part1:
            for v in part2:
                if random() < p:  # Conecta `u` y `v` con probabilidad `p`
                    bipartite_graph.add_edge([u, v])

        return bipartite_graph  # Devuelve el grafo bipartito generado

## Use of code

### Part 4 (Connected components via DFS)

In [ ]:
graph1 = Graph.random_graph(100, 1/25)
print(graph1)

1: 17 59 61 73 88
2: 5 10 24 35 79 88
3: 61 86
4: 24 69 83 94
5: 2 6 21 36 41 49 80 95 96
6: 5 77 84
7: 15 67 96
8: 12 35 49 74
9: 11 51 76 78
10: 2 51 60 81 87
11: 9 15 52 74
12: 8 13 46 56 98 100
13: 12 35 55 79 95
14: 33 38 48 55 79 81 84
15: 7 11 22 44 48 59 61 64 70 81 84 90 98
16: 29 83 87 99
17: 1 35 44 48 66
18: 59 64 95
19: 24
20: 27 55 72 76 90
21: 5 25 30 62 69 78 97
22: 15 39 65 100
23: 31 42
24: 2 4 19 28 52 57 65 90
25: 21 84
26: 36 61 77 87
27: 20 72
28: 24 43 55 82
29: 16 34 71 82
30: 21
31: 23 32 39 70 77 100
32: 31 42
33: 14 70 79
34: 29
35: 2 8 13 17 49 67 90
36: 5 26 54 94
37: 46
38: 14 69 92 94
39: 22 31 93 97
40: 46 53 64 68 74 81 83 85
41: 5
42: 23 32 61 89 97
43: 28 60 64 85 87
44: 15 17 68
45: 52
46: 12 37 40 53 59 95
47: 71 74
48: 14 15 17
49: 5 8 35 55 68 74 79 86 95
50: 75
51: 9 10 62 66 76 86
52: 11 24 45 62 83 93
53: 40 46 56 60 85 87
54: 36
55: 13 14 20 28 49 86 88
56: 12 53 63 67 70 80 85 86
57: 24 62 65 70 82 87 95
58:
59: 1 15 18 46 94
60: 10 43 53 85 

In [ ]:
print(len(graph1.connected_components()))
print(graph1.connected_components())

4
[{1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100}, {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 51, 52, 53, 54, 55, 56, 57, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100}, {50, 75}, {58}]


### Part 3 (Prufer codes)

In [ ]:
%%writefile tree1.txt
1 3
3 5
2 5
4 5
6 5

Writing tree1.txt


In [ ]:
tree1 = Graph.from_edges("tree1.txt")

In [ ]:
print(tree1)

1: 3
3: 1 5
5: 3 2 4 6
2: 5
4: 5
6: 5



In [ ]:
print(tree1.Prufer())

3 5 5 5


In [ ]:
print(Graph.tree_from_Prufer("3 5 5 5"))

1: 3
2: 5
3: 1 5
4: 5
5: 2 3 4 6
6: 5



In [ ]:
print(Graph.tree_from_Prufer("5 4 3 1"))

1: 3 6
2: 5
3: 4 1
4: 5 3
5: 2 4
6: 1



In [ ]:
print(Graph.tree_from_Prufer("1 1 1 2 3"))

1: 4 5 6 2
2: 1 3
3: 2 7
4: 1
5: 1
6: 1
7: 3



### Part2

In [ ]:
%%writefile edges.txt
a b
b c
b d
d
d c
e
f


Writing edges.txt


In [ ]:
%cat edges.txt

a b
b c
b d
d
d c
e
f


In [ ]:
graph2 = Graph.from_edges("edges.txt")
print(graph2)

a: b
b: a c d
c: b d
d: b c
e:
f:



In [ ]:
graph2.to_neighbourlist("neighbourhood.txt")

In [ ]:
%cat "neighbourhood.txt"

a: b
b: a c d
c: b d
d: b c
e:
f:


In [ ]:
!wget https://github.com/pgordin/GraphsSN2024_1/raw/main/weighted0.txt

--2024-11-07 14:30:48--  https://github.com/pgordin/GraphsSN2024_1/raw/main/weighted0.txt
Resolving github.com (github.com)... 20.27.177.113
Connecting to github.com (github.com)|20.27.177.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/pgordin/GraphsSN2024_1/main/weighted0.txt [following]
--2024-11-07 14:30:49--  https://raw.githubusercontent.com/pgordin/GraphsSN2024_1/main/weighted0.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 99 [text/plain]
Saving to: ‘weighted0.txt’

weighted0.txt       100%[===================>]      99  --.-KB/s    in 0s      

2024-11-07 14:30:49 (1.22 MB/s) - ‘weighted0.txt’ saved [99/99]



In [ ]:
graph3 = Graph.from_edges("weighted0.txt")
print(graph3)

A: B E
B: A C D
E: A D F H
C: B D F G
D: B C E F
F: C D E G H
G: C F H I
H: E F G
I: G



In [ ]:
# it works as well
!wget https://raw.githubusercontent.com/pgordin/GraphsSN2024_1/refs/heads/main/weighted0.txt

--2024-11-07 14:31:07--  https://raw.githubusercontent.com/pgordin/GraphsSN2024_1/refs/heads/main/weighted0.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 99 [text/plain]
Saving to: ‘weighted0.txt.1’

weighted0.txt.1     100%[===================>]      99  --.-KB/s    in 0s      

2024-11-07 14:31:07 (5.84 MB/s) - ‘weighted0.txt.1’ saved [99/99]



In [ ]:
graph4 = Graph.random_graph(10, 1/3)
print(graph4)

1: 2 4 6 7
2: 1 3 5 6 9
3: 2 9
4: 1 6
5: 2 6 10
6: 1 2 4 5
7: 1 9 10
8: 10
9: 2 3 7
10: 5 7 8



In [ ]:
print(Graph.cycle(6))

1: 2 6
2: 1 3
3: 2 4
4: 3 5
5: 4 6
6: 5 1



### Part1

In [ ]:
vertices = ["a", "b", "c", "d"]
matrix = np.array([[0,1,0,0],[1,0,1,0],[0,1,0,1],[0,0,0,1]])
print(vertices)
print(matrix)
print("---------------------------")
print_matrix(vertices, matrix)
print("---------------------------")
print_matrix(None,matrix)

['a', 'b', 'c', 'd']
[[0 1 0 0]
 [1 0 1 0]
 [0 1 0 1]
 [0 0 0 1]]
---------------------------
a :  b
b :  a  c
c :  b  d
d :  d
---------------------------
1 :  2
2 :  1  3
3 :  2  4
4 :  4


In [ ]:
graph_dict = {
  "a": ["b"],
  "b": ["a", "c"],
  "c": ["b", "d"],
  "d": ["c"]
}
print(graph_dict)
print("---------------------------")
print_dict(graph_dict)

{'a': ['b'], 'b': ['a', 'c'], 'c': ['b', 'd'], 'd': ['c']}
---------------------------
a :  b
b :  a  c
c :  b  d
d :  c


In [ ]:
graph1 = Graph.from_matrix(matrix, vertices)
print(graph1)

a: b
b: a c
c: b d
d: d



In [ ]:
print(Graph(graph_dict))  # the same result

a: b
b: a c
c: b d
d: c



In [ ]:
print(graph1.vertices())

['a', 'b', 'c', 'd']


In [ ]:
print(graph1.matrix())

[[0 1 0 0]
 [1 0 1 0]
 [0 1 0 1]
 [0 0 0 1]]


In [ ]:
graph1.add_vertex("e")
print(graph1)

a: b
b: a c
c: b d
d: d
e:



In [ ]:
graph1.add_edge(["e", "f"])
print(graph1)

a: b
b: a c
c: b d
d: d
e: f
f: e



In [ ]:
graph1.add_arc(["e", "a"])  # breaking the symmetry
print(graph1)

a: b
b: a c
c: b d
d: d
e: f a
f: e



In [ ]:
graph1.add_edge(["e", "a"])  # restoring the symmetry
print(graph1)

a: b e
b: a c
c: b d
d: d
e: f a
f: e



In [ ]:
graph1.add_edge(["e", "f"]) # do nothing, an edge already exists
print(graph1)

a: b e
b: a c
c: b d
d: d
e: f a
f: e



In [ ]:
graph1.del_vertex("f")  # removing a vertex
print(graph1)

a: b e
b: a c
c: b d
d: d
e: a



In [ ]:
graph1.add_edge(["e", "e"]) # an error

ValueError: Loops are not allowed!

In [ ]:
graph1.add_arc(["e", "e"]) # OK - loops are allowed in digraphs
print(graph1)

a: b e
b: a c
c: b d
d: d
e: a e

